<a href="https://colab.research.google.com/github/chiyanglin-AStar/science_coding/blob/main/qiskit_1_0_example1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## qiskit 1.0 [example1](https://pypi.org/project/qiskit/)

In [1]:
!pip install qiskit
!pip install qiskit[visualization]
!pip install qiskit[machine-learning]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=39f7fe0f9e1f39bf4af2e672b73e0560fcdc49a80f21491a67699bf90e73e424
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


This simple example makes an entangled state known as a GHZ state
. It uses the standard quantum gates: Hadamard gate (h), Phase gate (p), and CNOT gate (cx).

Once you've made your first quantum circuit, choose which primitive function you will use. Starting with sampler, we use measure_all(inplace=False) to get a copy of the circuit in which all the qubits are measured:

In [2]:
import numpy as np
from qiskit import QuantumCircuit

# 1. A quantum circuit for preparing the quantum state |000> + i |111>
qc_example = QuantumCircuit(3)
qc_example.h(0)          # generate superpostion
qc_example.p(np.pi/2,0)  # add quantum phase
qc_example.cx(0,1)       # 0th-qubit-Controlled-NOT gate on 1st qubit
qc_example.cx(0,2)       # 0th-qubit-Controlled-NOT gate on 2nd qubit

In [3]:
# 2. Add the classical output in the form of measurement of all qubits
qc_measured = qc_example.measure_all(inplace=False)

# 3. Execute using the Sampler primitive
from qiskit.primitives.sampler import Sampler
sampler = Sampler()
job = sampler.run(qc_measured, shots=1000)
result = job.result()
print(f" > Quasi probability distribution: {result.quasi_dists}")

 > Quasi probability distribution: [{0: 0.502, 7: 0.498}]


<ipython-input-3-a33bc51d07f9>:6: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()


Running this will give an outcome similar to {0: 0.497, 7: 0.503} which is 000 50% of the time and 111 50% of the time up to statistical fluctuations. To illustrate the power of Estimator, we now use the quantum information toolbox to create the operator
 and pass it to the run() function, along with our quantum circuit. Note the Estimator requires a circuit without measurement, so we use the qc_example circuit we created earlier.

In [4]:
# 2. Define the observable to be measured
from qiskit.quantum_info import SparsePauliOp
operator = SparsePauliOp.from_list([("XXY", 1), ("XYX", 1), ("YXX", 1), ("YYY", -1)])

# 3. Execute using the Estimator primitive
from qiskit.primitives import Estimator
estimator = Estimator()
job = estimator.run(qc_example, operator, shots=1000)
result = job.result()
print(f" > Expectation values: {result.values}")

 > Expectation values: [4.]


<ipython-input-4-d56052fa4968>:7: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


Running this will give the outcome 4. For fun, try to assign a value of +/- 1 to each single-qubit operator X and Y and see if you can achieve this outcome. (Spoiler alert: this is not possible!)

Using the Qiskit-provided qiskit.primitives.Sampler and qiskit.primitives.Estimator will not take you very far. The power of quantum computing cannot be simulated on classical computers and you need to use real quantum hardware to scale to larger quantum circuits. However, running a quantum circuit on hardware requires rewriting to the basis gates and connectivity of the quantum hardware. The tool that does this is the transpiler, and Qiskit includes transpiler passes for synthesis, optimization, mapping, and scheduling. However, it also includes a default compiler, which works very well in most examples. The following code will map the example circuit to the basis_gates = ['cz', 'sx', 'rz'] and a linear chain of qubits
 with the coupling_map =[[0, 1], [1, 2]].

In [5]:
from qiskit import transpile
qc_transpiled = transpile(qc_example, basis_gates = ['cz', 'sx', 'rz'], coupling_map =[[0, 1], [1, 2]] , optimization_level=3)

Executing your code on real quantum hardware
Qiskit provides an abstraction layer that lets users run quantum circuits on hardware from any vendor that provides a compatible interface. The best way to use Qiskit is with a runtime environment that provides optimized implementations of sampler and estimator for a given hardware platform. This runtime may involve using pre- and post-processing, such as optimized transpiler passes with error suppression, error mitigation, and, eventually, error correction built in. A runtime implements qiskit.primitives.BaseSampler and qiskit.primitives.BaseEstimator interfaces. For example, some packages that provide implementations of a runtime primitive implementation are:

https://github.com/Qiskit/qiskit-ibm-runtime
Qiskit also provides a lower-level abstract interface for describing quantum backends. This interface, located in qiskit.providers, defines an abstract BackendV2 class that providers can implement to represent their hardware or simulators to Qiskit. The backend class includes a common interface for executing circuits on the backends; however, in this interface each provider may perform different types of pre- and post-processing and return outcomes that are vendor-defined. Some examples of published provider packages that interface with real hardware are:

https://github.com/qiskit-community/qiskit-ionq
https://github.com/qiskit-community/qiskit-aqt-provider
https://github.com/qiskit-community/qiskit-braket-provider
https://github.com/qiskit-community/qiskit-quantinuum-provider
https://github.com/rigetti/qiskit-rigetti
You can refer to the documentation of these packages for further instructions on how to get access and use these systems.

Contribution Guidelines
If you'd like to contribute to Qiskit, please take a look at our contribution guidelines. By participating, you are expected to uphold our code of conduct.

We use GitHub issues for tracking requests and bugs. Please join the Qiskit Slack community for discussion, comments, and questions. For questions related to running or using Qiskit, Stack Overflow has a qiskit. For questions on quantum computing with Qiskit, use the qiskit tag in the Quantum Computing Stack Exchange (please, read first the guidelines on how to ask in that forum).

Authors and Citation
Qiskit is the work of many people who contribute to the project at different levels. If you use Qiskit, please cite as per the included BibTeX file.

Changelog and Release Notes
The changelog for a particular release is dynamically generated and gets written to the release page on Github for each release. For example, you can find the page for the 0.46.0 release here:

https://github.com/Qiskit/qiskit/releases/tag/0.46.0

The changelog for the current release can be found in the releases tab: Releases The changelog provides a quick overview of notable changes for a given release.

Additionally, as part of each release, detailed release notes are written to document in detail what has changed as part of a release. This includes any documentation on potential breaking changes on upgrade and new features. See all release notes here.

Acknowledgements
We acknowledge partial support for Qiskit development from the DOE Office of Science National Quantum Information Science Research Centers, Co-design Center for Quantum Advantage (C2QA) under contract number DE-SC0012704.

License
Apache License 2.0